# TensorFlow BasicsTutorial with MNIST DataSet

TensorFlow is a powerful library for doing large-scale numerical computation. One of the tasks at which it excels is implementing and training deep neural networks

### Importing the Libraries

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_mldata

In [2]:
mnist1= fetch_mldata('MNIST original')

In [3]:
#print(mnist1)

### Forking MNIST dataset

These two lines are used to fork the mnist data set in our code
Here mnist is a class now, which stores the training, validation and testing sets as NumPy arrays.

mnist data is available in tensorflow examples. It is read by input_data.read_data_sets function and is present as MNIST_data. Here, the labels are prespecified to be onehot arrays

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


### Starting TF interactive session

Tensorflow uses C++ to do its computation in the backend. This is because it is highly efficient and faster than python. Thus, this connection of Tensorflow in python to its backend C++ functions is known as Session. 

Generally, we first make a data flow graph in tensorflow and then launch it in a session, passing it the required placeholders or just evaluating it.

We use InteractiveSession class which allows us to interleave/parellelize(allows us to make changes even after running the session) the operations which build a computational graph with one that runs the graph otherwise, we have to build the entire the computational graph before starting the session and launching/firing the graph.


In [5]:
sess = tf.InteractiveSession()

### Computational Graph

We use lbraries like Numpy in python which does extensive numerical computation ouside python using some other languages(like C++). But this switching of values causes overhead processing which is bad for computations on GPUs or in a distributed manner computation due to high transferring costs.

Tensorflow also computes externally but instead of running a single expensive operation independentlyfrom python, it lets us describe a graph of interacting operations that run entirely outside python which avoids this overhead cost.

## Softmax Regression Model

We make a regression with a single linear layer.

### Placeholders

x is the input and y_ is the labels. These are the nodes in the computational graph for input images and target output classes/one hot labels. x and y_ are values which we we'll input when we ask tensorflow to run a computation.

MNIST has images of shape 28X28 which has been flattened to one dimension, hence having 784 input features represented by each pixel. The first dimension represents the batch size here. None presents a sence of variability and indicates that the batch can be of any size.
The target output/labels, y_, will also consist of a 2d tensor where each row will be a 10 dimensional vector. The output classes inthe dataset ranges from 0 to 9.

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

### Variables

Weights and biases here are the variables. Unlike placeholders, they do not interact with the outside world. Rather, they are those values that "lives in tensorflow's computational graph". They can be used and modified by computation.

tf.Variable() method creates the variables and initializes them(0s here) W is a 784X10matrix(weight matrix shape= 784 inputs and 10 outputs{single layered}) and b is 10 dimensional vector as we have 10 classes(10 neurons in the single layer)

In [7]:
W = tf.Variable(tf.zeros([784,10]))#input to output dimensions of the weight matrix
b = tf.Variable(tf.zeros([10]))

Before the variables can be used in a session, they need to be initialized in the session with their initial value which can be done using sess.run(variable)command. To initialize all of them at once, we use tf.global_variables_initializer() function to point to all the variables. It returns an Output that initializes global variables.

In [8]:
sess.run(tf.global_variables_initializer())

### Forward Pass

In [9]:
y = tf.matmul(x,W) + b

### Predicted class and loss(cost) function 

Scores in the final layer without applying softmax are called logit. We provide it the output of the previous layer without applying softmax to it(z) as its input and it returns the cross entropy as its output. 

tf,reduce_mean averages all the elements of a tensor, and hence calculates the mean cost over all the images.

tf.nn.softmax_cross_entropy_with_logits Computes softmax cross entropy between `logits` and `labels`. Measures the probability error in discrete classification tasks in which the
classes are mutually exclusive. It returns  a 1-D `Tensor` of length `batch_size` of the same type as `logits` with the softmax cross entropy loss.

In other words tf.nn.softmax_cross_entropy_with_logits internally applies the softmax on the model's unnormalized model prediction and sums across all classes, and tf.reduce_mean takes the average over these sums.


In [10]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

### Training the model/ The Backward Pass

As the Tensorflow knows the wholle computational graph, it can use automatic differentiation to find the gradients(delL,dell,dcdw,dcdb). Here , this uses a gradient descent optimzer and trains the model according to minimizing the cross entropy(cost) of the whole system. It does so by adding operators in the model flow graph to compute cost gradients(delL, dell, dc/dw,dc/db, to compute the parameter updates-> wl=wl-alpha*(dc/dw) and bl=bl-alpha*(dc/db). 
Also, alpha here is learning rate

In [11]:
alpha= tf.Variable(0.5)
sess.run(tf.global_variables_initializer())

In [12]:
train_step = tf.train.GradientDescentOptimizer(alpha).minimize(cross_entropy)

In [13]:
#?tf.train.GradientDescentOptimizer().minimize()

The returned operation train_step, when run, will apply the gradient descent updates to the parameters. Training the model can therefore be accomplished by repeatedly running train_step.

The for loop here represents iterations we are performing. Here, we load a number of training examples in each iteration using mnist.train.next_batch(100) and store this value in batch. The 0the position in batch consists of the datapoints/images and the 1st positions in batch consists of the corresponding labels/target classes. This is fed to the placeholder x and y_ from batch using the feed_dict() function. 

We finally run the train step function in the session. Running any step in the session requires feeding of the placeholders. All the operations required to get to the point which we ran in the graph, are first executed. Hence, in a way, the whole graph gets executed one by one till the backwardpass itself. It feeds the input to x and labels to y_.First of all, the weights and biases gets initialized as forward pass requires them. The forward pass runs then, computing the output which is required by the cost computing function. According to this, the cross entropy is calculated as it is needed by the backward pass to minimize, and hence, the final step, train_step is run.

The whole process is repeated several times according to the for loop argument

In [24]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
   

tf.argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis. We can use tf.equal to check if our prediction matches the truth. This gives a array of booleans whose sum could be use to calculate total number of true positives. Casting this gives our accuracy.

In [25]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [26]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Evaluation

as soon as we run our .eval function of any valriable or stp in the model graph, it runs all the preceeding steps reuired to compute that

In [27]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9215


In [28]:
print((x.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})).shape)

(10000, 784)
